In [1]:
#loading required libraries
import pandas as pd
import numpy as numpy

In [3]:
# loading required dataset

# MODIS Dataset
bauchi_modis=pd.read_csv('Bauchi-MODIS.csv')
plateau_modis= pd.read_csv('Plateau-MODIS.csv')

# VIIRS Dataset
bauchi_viirs = pd.read_csv('Bauchi-VIIRS.csv')
plateau_viirs = pd.read_csv('Plateau-VIIRS.csv')

## Understanding the data

### Region: Bauchi

In [8]:
bauchi_modis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   system:time_start  549 non-null    object 
 1   NDVI               549 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.7+ KB


In [9]:
bauchi_viirs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   system:time_start  550 non-null    object 
 1   NDVI               550 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.7+ KB


- Both datasets do not contain missing values.
- The date is an object datatype

### Region: Plateau